<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/BCELoss/TestAndTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [0]:
import os
import logging
import sys

In [0]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone -b BCELoss https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'

# Import packages

In [0]:
from DatasetCIFAR.data_set import Dataset
from DatasetCIFAR import ResNet
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import Subset, DataLoader

# Hyper-Parameters

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'
BATCH_SIZE = 128
NUM_WORKERS = 100
TASK_SIZE = 10
NUM_EPOCHS = 70
WEIGHT_DECAY = 0.00001
LR = 2
STEP_SIZE = [49,63]
GAMMA = 1/5

# Define Network

In [0]:
resNet = ResNet.resnet32(num_classes=100)
resNet = resNet.to(DEVICE)

In [0]:
resnet_transformer = transforms.Compose([transforms.Resize(32), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])


# Define DataSets

In [0]:
trainDS = Dataset(train=True, transform = resnet_transformer)
testDS = Dataset(train=False, transform = resnet_transformer)

Files already downloaded and verified
Files already downloaded and verified


# Train phase

In [0]:
def plotEpoch(pars):
  import numpy as np
  import matplotlib.pyplot as plt

  x_epochs = np.linspace(1,NUM_EPOCHS,NUM_EPOCHS)
  y1 = [e[0] for e in pars] #val acuracy
  y2 = [e[2] for e in pars] #train accuracy
  plt.plot(x_epochs, y1 , '-', color='red')
  plt.plot(x_epochs, y2, '-', color='blue')
  plt.xlabel("Epoch")
  plt.legend(['Validation Accuracy', 'Train accuracy'])
  plt.show()

  y1 = [e[1] for e in pars] #val loss
  y2 = [e[3] for e in pars] #train loss
  plt.plot(x_epochs, y1 , '-', color='red')
  plt.plot(x_epochs, y2, '-', color='blue')
  plt.xlabel("Epoch")
  plt.legend(['Validation Loss', 'Train Loss'])
  plt.show()



In [0]:
def plotTask():
  import numpy as np
  import matplotlib.pyplot as plt

  x_tasks =  np.linspace(10, 100, 10)

  plt.plot(x_tasks, y1 , '-', color='black')
  plt.xlabel("Epoch")
  plt.legend(['Validation Accuracy'])
  plt.show()



In [0]:
def eachepochevaluation():
  resNet.train(False) # Set Network to evaluation mode
  running_corrects = 0
  #confusion_matrix = torch.zeros(10, 10)
  for images, labels in test_loader:
    images = images.float().to(DEVICE)
    labels = labels.to(DEVICE)
    onehot_labels = torch.eye(100)[labels].to(DEVICE)
    # Forward Pass
    outputs = resNet(images)
    # Get predictions
    _, preds = torch.max(outputs.data, 1)
    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(test_dataset))
  
  #Calculate Loss
  loss = criterion(outputs,onehot_labels)
  print("epoch =" + str(epoch))
  print('Validation Loss: {} Validation Accuracy : {}'.format(loss,accuracy))
  return (accuracy, loss.item())

In [0]:
pars_epoch = []
pars_tasks = []
test_indexes = []

for task in range(0, 100, TASK_SIZE):
  criterion = nn.BCEWithLogitsLoss()
  optimizer = torch.optim.SGD(resNet.parameters(), lr=2., weight_decay=WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.MultiStepLR(optimizer, STEP_SIZE, gamma=GAMMA)

  n_classes = task + TASK_SIZE

  resNet.train(True)
  print("Task classes {} to {}".format(task, n_classes))

  train_indexes = trainDS.__getIndexesGroups__(task)
  test_indexes = test_indexes + testDS.__getIndexesGroups__(task)

  train_dataset = Subset(trainDS, train_indexes)
  test_dataset = Subset(testDS, test_indexes)

  train_loader = DataLoader( train_dataset, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE)
  test_loader = DataLoader( test_dataset, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE )

  current_step = 0
  for epoch in range(NUM_EPOCHS):
    scheduler.step()
    print(scheduler.get_lr(), "   ", scheduler.get_last_lr())
    running_corrects = 0
    for images, labels in train_loader:
      
      # Bring data over the device of choice
      images = images.float().to(DEVICE)
      labels = labels.to(DEVICE)

      onehot_labels = torch.eye(100)[labels].to(DEVICE)

      optimizer.zero_grad() # Zero-ing the gradients

      # Forward pass to the network
      outputs = resNet(images)
      
      #loss = criterion(outputs,labels)
      loss = criterion(outputs,onehot_labels)

      # Get predictions
      _, preds = torch.max(outputs.data, 1)
      
      # Update Corrects
      running_corrects += torch.sum(preds == labels.data).data.item()
      
      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients

      current_step += 1
    # Calculate Accuracy
    accuracy = running_corrects / float(len(train_dataset))
    print("At step ", str(task), " and at epoch = ", epoch, " the loss is = ", loss.item(), " and accuracy is = ", accuracy)
    param=eachepochevaluation()
    pars_epoch.append( (param[0], param[1], accuracy, loss.item()) )
    #pars_epoch -->   val_acc,  val_loss, train_acc,train_loss
    pars_tasks[task/10] += val_acc
  
  plotting(pars_epoch)
  pars_epoch = []
  pars_tasks[task/10] /= NUM_EPOCHS
plotTask(pars_tasks)

Task classes 0 to 10
Step 0, Loss 2.5169520378112793
Step 10, Loss 1.8353420495986938
Step 20, Loss 1.7539255619049072
Step 30, Loss 1.658636212348938
At step  0  and at epoch =  0  the loss is =  2.214704990386963  and accuracy is =  0.3648
Validation Accuracy: 0.437 Validation Loss : 1.850717544555664
Step 40, Loss 1.887252688407898
Step 50, Loss 2.27921724319458
Step 60, Loss 2.2522501945495605
Step 70, Loss 2.3270435333251953
At step  0  and at epoch =  1  the loss is =  2.4119765758514404  and accuracy is =  0.1494
Validation Accuracy: 0.132 Validation Loss : 2.3367741107940674
Step 80, Loss 2.329252243041992
Step 90, Loss 2.1811375617980957
Step 100, Loss 2.195904493331909
Step 110, Loss 2.230470895767212
At step  0  and at epoch =  2  the loss is =  2.379544258117676  and accuracy is =  0.1982
Validation Accuracy: 0.259 Validation Loss : 2.0411736965179443
Step 120, Loss 2.1440718173980713
Step 130, Loss 2.0956485271453857
Step 140, Loss 1.974965214729309
Step 150, Loss 2.117064